In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3

In [0]:
import torchtext
from torchtext import data
from torchtext import datasets
from torchtext.vocab import GloVe
from torchtext.vocab import FastText
from torchtext.vocab import Vectors

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch
from torch.autograd import Variable

import pickle

import numpy as np
from keras.utils import np_utils

import re, os
from glob import glob
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import linecache
import pandas as pd

In [0]:
import MeCab
import re

tagger = MeCab.Tagger("-Owakati")

def make_wakati(sentence):
    sentence = tagger.parse(sentence)
    sentence = re.sub(r'[0-9０-９a-zA-Zａ-ｚＡ-Ｚ]+', " ", sentence)
    sentence = re.sub(r'[\．_－―─！＠＃＄％＾＆\-‐|\\＊\“（）＿■×+α※÷⇒—●★☆〇◎◆▼◇△□(：〜～＋=)／*&^%$#@!~`){}［］…\[\]\"\'\”\’:;<>?＜＞〔〕〈〉？、。・,\./『』【】「」→←○《》≪≫\n\u3000]+', "", sentence)
    wakati = sentence.split(" ")
    wakati = list(filter(("").__ne__, wakati))
    return wakati

In [0]:
# import nltk
# from nltk import stem
# nltk.download('all')

# def nltk_analyzer(text):
#   stemmer = stem.LancasterStemmer()
#   text = re.sub(re.compile(r'[!-\/:-@[-`{-~]'), ' ', text)
#   text = stemmer.stem(text)
#   text = text.replace('\n', '') # 改行削除
#   text = text.replace('\t', '') # タブ削除
#   morph = nltk.word_tokenize(text)
#   return morph

In [0]:
drive_dir = "drive/My Drive/Colab Notebooks/rte/"
datasets = pd.read_csv(drive_dir + 'Translation_of_SNLI.csv')
train_df, test_df = train_test_split(datasets, train_size=0.95)

col = ["sentence1", "sentence2", "gold_id"]

labels = ["contradiction", "entailment", "neutral", "-"]




# def concat_df(df):
#   label_ids = []
#   for label in df['gold_label']:
#     idx = labels.index(label)
#     label_ids.append(idx)
#   join_df = pd.DataFrame(label_ids, columns=['gold_id'])
#   print(len(df), len(join_df))
#   return pd.concat([df, join_df], axis=1)

# train_df = concat_df(train_df)
# test_df = concat_df(test_df)

# rm y label "-" line and fillna
# train_df = train_df[train_df["gold_label"] != "-"].fillna("")
# test_df = test_df[test_df["gold_label"] != "-"].fillna("")

In [71]:
train_df

,Unnamed: 0,gold_label,t1_en,t2_en,t1_jp,t2_jp
162176,162176,neutral,A man in a white jacket walks underneath a bri...,He is homeless and trying to get to his box to...,白いジャケットを着た男性が橋の下を歩き、光線で強調されます。,彼はホームレスで、自分の箱に寝ようとしています。
253354,253354,entailment,Three people look up at a man on a ladder agai...,hree people look up at a man on a ladder.,3人がレンガの壁に向かってはしごの男を見上げます。,3人がはしごの男を見上げます。
106650,106650,entailment,A group of people on a crowded sidewalk boardi...,A group of people are getting on a bus.,2階建てバスに乗る混雑した歩道上の人々のグループ。,人々のグループがバスに乗っています。
373023,373023,contradiction,"Vendors in Europe, a man and a woman preparing...",A man and woman are selling fruits and vegetab...,ヨーロッパのベンダー、野菜と果物を販売する日のために準備する男女。,男性と女性がアメリカで果物と野菜を販売しています。
124924,124924,contradiction,The little girl is picking up the beans.,a little girl throws away some food,小さな女の子は豆を拾っています。,小さな女の子が食べ物を捨てる
...,...,...,...,...,...,...
199449,199449,entailment,"Many people gathered outside, sitting on bench...","Many people gathered outside, sitting on bench...",多くの人々が外に集まり、ベンチや草の中に座っていました。,多くの人々が外に集まり、外のベンチに座っていました。
55746,55746,entailment,A man with a race number on his shirt running ...,There is a man running,ベビーカーで小さな女の子と実行している彼のシャツにレース番号を持つ男。,走っている男がいる
268541,268541,contradiction,Blond girl paying with a hula hoop.,The girls are taking a math test.,フラフープで支払うブロンドの女の子。,女の子は数学のテストを受けています。
503362,503362,entailment,A craftsman thoughtfully considers his designs...,A craftsman is thinking about his designs.,職人は、ワークショップで自分のデザインを熟考します。,職人は自分のデザインについて考えています。


In [0]:
drive_dir = "drive/My Drive/Colab Notebooks/rte/"
train_df = pd.read_csv(drive_dir + "snli_1.0_train.txt", sep="\t", header=0)
val_df = pd.read_csv(drive_dir + "snli_1.0_dev.txt", sep="\t", header=0)
test_df = pd.read_csv(drive_dir + "snli_1.0_test.txt", sep="\t", header=0)

col = ["sentence1", "sentence2", "gold_id"]

labels = ["contradiction", "entailment", "neutral", "-"]

def concat_df(df):
  label_ids = []
  for label in df['gold_label']:
    idx = labels.index(label)
    label_ids.append(idx)
  join_df = pd.DataFrame(label_ids, columns=['gold_id'])
  return pd.concat([df, join_df], axis=1)

train_df = concat_df(train_df)
val_df = concat_df(val_df)
test_df = concat_df(test_df)

# rm y label "-" line and fillna
train_df = train_df[train_df["gold_label"] != "-"].fillna("")
val_df = val_df[val_df["gold_label"] != "-"].fillna("")
test_df = test_df[test_df["gold_label"] != "-"].fillna("")

train_df[col].to_csv(drive_dir + "train.tsv", sep="\t", index=None, header=False)
val_df[col].to_csv(drive_dir + "val.tsv", sep="\t", index=None, header=False)
test_df[col].to_csv(drive_dir + "test.tsv", sep="\t", index=None, header=False)

In [0]:
TEXT = data.Field(sequential=True, tokenize=make_wakati, lower=True, include_lengths=True, batch_first=True)
LABEL = data.Field(sequential=False, use_vocab=False, is_target=True)
  
train, val, test = data.TabularDataset.splits(
      path=drive_dir, train='train.tsv',
      validation='val.tsv', test='test.tsv', format='tsv',
      fields=[('Text1', TEXT), ('Text2', TEXT), ('Label', LABEL)])

In [0]:
glove_vectors = Vectors(name=drive_dir + "glove.6B.200d.txt")
TEXT.build_vocab(train, val, test, vectors=glove_vectors, min_freq=1)

100%|█████████▉| 399404/400000 [00:34<00:00, 11775.11it/s]

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_iter, val_iter, test_iter = data.Iterator.splits((train, val, test), batch_sizes=(100, 100, 100), device=device, repeat=False, sort=False)

In [0]:
TEXT.vocab.vectors.size()[0]

34463

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class LSTMEmbedding(nn.Module):
  def __init__(self, embedding_dim, lstm_dim, vocab_size):
    super(LSTMEmbedding, self).__init__()
    self.lstm_dim = lstm_dim
    self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
    self.word_embeddings.weight.data.copy_(TEXT.vocab.vectors)
    self.word_embeddings.requires_grad_ = False    
    self.lstm = nn.LSTM(embedding_dim, lstm_dim, batch_first=True, dropout=0.5)
  
  def forward(self, text):
    embeds = self.word_embeddings(text)
    _, state = self.lstm(embeds)
    return state[0]


class MergeLSTM(nn.Module):
  def __init__(self, lstm_dim, hidden_dim, tagset_size):
    super(MergeLSTM, self).__init__()
    self.lstm_dim = lstm_dim
    self.hidden_dim = hidden_dim
    self.tagset_size = tagset_size
    self.softmax = nn.Softmax(dim=1)
    self.main = nn.Sequential(
        nn.BatchNorm1d(self.lstm_dim*2),

        nn.Linear(self.lstm_dim*2, self.hidden_dim),
        nn.PReLU(),
        nn.Dropout(0.5),
        nn.BatchNorm1d(self.hidden_dim),
        
        nn.Linear(self.hidden_dim, self.hidden_dim),
        nn.PReLU(),
        nn.Dropout(0.5),
        nn.BatchNorm1d(self.hidden_dim),

        nn.Linear(self.hidden_dim, self.hidden_dim),
        nn.PReLU(),
        nn.Dropout(0.5),
        nn.BatchNorm1d(self.hidden_dim),
        
        nn.Linear(self.hidden_dim, tagset_size)
    )

    
  def forward(self, state1, state2):
    state = torch.cat([state1, state2], dim=1)
    hidden = self.main(state)
    tag_score = self.softmax(hidden)
    return tag_score

EMBEDDING_DIM = 200
LSTM_DIM = 200
HIDDEN_DIM = 300
VOCAB_SIZE = TEXT.vocab.vectors.size()[0]
TAG_SIZE = 3

# to(device)でモデルがGPU対応する
model1 = LSTMEmbedding(EMBEDDING_DIM, LSTM_DIM, VOCAB_SIZE).to(device)
model2 = LSTMEmbedding(EMBEDDING_DIM, LSTM_DIM, VOCAB_SIZE).to(device)
merge = MergeLSTM(LSTM_DIM, HIDDEN_DIM, TAG_SIZE).to(device)

# loss_function = nn.NLLLoss()
loss_function = nn.CrossEntropyLoss()
model1_optimizer = optim.Adam(model1.parameters(), lr=0.001)
model2_optimizer = optim.Adam(model2.parameters(), lr=0.001)
merge_optimizer = optim.Adam(merge.parameters(), lr=0.001)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:51: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [0]:
losses = []
for epoch in range(100):
  all_loss = 0

  for idx, batch in enumerate(train_iter):
    batch_loss = 0
    model1.zero_grad()
    model2.zero_grad()
    merge.zero_grad()
    
    text1_tensor = batch.Text1[0]
    text2_tensor = batch.Text2[0]
    label_tensor = batch.Label

    state1 = model1(text1_tensor).squeeze()
    state2 = model2(text2_tensor).squeeze()
    score = merge(state1, state2)

    batch_loss = loss_function(score, label_tensor)
    batch_loss.backward()
    model1_optimizer.step()
    model2_optimizer.step()
    merge_optimizer.step()
    
    all_loss += batch_loss.item()
  print("epoch", epoch, "\t" , "loss", all_loss)

epoch 0 	 loss 4996.707608103752
epoch 1 	 loss 4735.979795217514
epoch 2 	 loss 4603.145826458931
epoch 3 	 loss 4485.0309846401215
epoch 4 	 loss 4394.917196214199
epoch 5 	 loss 4333.4020909667015
epoch 6 	 loss 4281.546170771122
epoch 7 	 loss 4239.057874262333
epoch 8 	 loss 4200.856141209602
epoch 9 	 loss 4169.181241154671
epoch 10 	 loss 4144.0787162184715
epoch 11 	 loss 4119.282353520393
epoch 12 	 loss 4098.843482255936
epoch 13 	 loss 4082.2214800715446
epoch 14 	 loss 4063.7675580978394
epoch 15 	 loss 4050.0937114953995
epoch 16 	 loss 4032.4641233086586
epoch 17 	 loss 4020.55526971817
epoch 18 	 loss 4013.0062601566315
epoch 19 	 loss 4000.9986991882324
epoch 20 	 loss 3987.029092490673
epoch 21 	 loss 3977.382672905922
epoch 22 	 loss 3973.0348151922226
epoch 23 	 loss 3963.71888846159
epoch 24 	 loss 3955.0785363316536


KeyboardInterrupt: ignored

In [0]:
test_num = len(test)
a = 0
with torch.no_grad():
  for idx, batch in enumerate(test_iter):
    text1_tensor = batch.Text1[0]
    text2_tensor = batch.Text2[0]
    label_tensor = batch.Label
    
    state1 = model1(text1_tensor).squeeze()
    state2 = model2(text2_tensor).squeeze()
    score = merge(state1, state2)
    
    _, pred = torch.max(score, 1)
    for j, ans in enumerate(label_tensor):
      if pred[j].item() == ans.item():
        a += 1

print("predict : ", a / test_num)

predict :  0.7398208469055375
